# The Julia Express

### Bogumił Kamiński (http://bogumilkaminski.pl)

## Contents
[Introduction](#introduction)

[Getting around](#gettingaround)

[Basic literals and types](#basicliterals)

[Complex literals and types](#complexliterals)

[Strings](#strings)

[Programming constructs](#programming)

[Variable scoping](#scoping)

[Modules](#modules)

## <a name="introduction"></a>Inroduction
The Purpose of this document is to introduce programmers to Julia programming by example. This is a simplified exposition of the language.

If some packages are missing on your system use Pkg.add to require installing them. There are many add-on packages
which you can browse at http://pkg.julialang.org/.

Major stuff not covered (please see the documentation):

1. parametric types;
2. parallel and distributed processing;
3. advanced I/O operations;
4. package management; see `Pkg`;
5. interaction with system shell; see `run`;
6. exception handling; see `try`;
7. creation of coroutines; see `Task`;
8. two-way integration with C and Fortran.

You can find current Julia documentation at http://julia.readthedocs.org/en/latest/manual/.

The code was executed using the following Julia version:

In [1]:
versioninfo()

Julia Version 0.4.5
Commit 2ac304d (2016-03-18 00:58 UTC)
Platform Info:
  System: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Core(TM) i5-5200U CPU @ 2.20GHz
  WORD_SIZE: 64
  BLAS: libopenblas (USE64BITINT DYNAMIC_ARCH NO_AFFINITY Haswell)
  LAPACK: libopenblas64_
  LIBM: libopenlibm
  LLVM: libLLVM-3.3


Remember that you can expect every major version of Julia to introduce breaking changes.

Check https://github.com/JuliaLang/julia/blob/master/NEWS.md for release notes.

All sugestions how this guide can be improved are welcomed. Please contact me at [bkamins@sgh.waw.pl](mailto:bkamins@sgh.waw.pl).

## <a name="gettingaround"></a>Getting around

Running julia invokes interactive (REPL) mode. In this mode some useful commands are:

1. ^D (exits Julia, also calling `exit(c)` quits with exit code `c`);
2. ^C (interrupts computations);
3. ? (enters help mode)
4. ; (enters system shell mode)
5. putting ; after the expression will disable showing of its value.

Examples of some essential functions in REPL (they can be also invoked in scripts):

In [2]:
apropos("apropos") # search documentation for "apropos" string

apropos
Base.Docs.stripmd


In [3]:
@edit max(1,2) # show the definition of max function when invoked with arguments 1 and 2 in text editor

Unknown editor: no line number information passed.
The method is defined at line 239.


In [4]:
whos() # list of global variables and their types

                          Base  24865 KB     Module
                        Compat    223 KB     Module
                          Core   2972 KB     Module
                        IJulia    285 KB     Module
                IPythonDisplay     27 KB     Module
                          JSON    208 KB     Module
                          Main  28359 KB     Module
                        Nettle     60 KB     Module
                           ZMQ     84 KB     Module


In [10]:
cd("D:/") # change working directory to D:/ (on Windows)
pwd() # get current working directory

"D:\\"

In [7]:
include("file.jl") # execute source file, LoadError if execution fails

LoadError: LoadError: could not open file D:\file.jl
while loading In[7], in expression starting on line 1

In [11]:
clipboard([1,2]) # copy data to system clipboard
clipboard() # load data from system clipboard as string

"[1,2]"

In [12]:
workspace() # clear worskspace - create new Main module (only to be used interactively)

You can execute Julia script by running `julia script.jl`.

Try saving the following example script to a file and run it (more examples of all the constructs used are given in
following sections):

In [13]:
"Sieve of Eratosthenes function docstring"
function es(n::Int) # accepts one integer argument
    isprime = ones(Bool, n) # n-element vector of true-s
    isprime[1] = false # 1 is not a prime
    for i in 2:round(Int, sqrt(n)) # loop integers from 2 to sqrt(n)
        if isprime[i] # conditional evaluation
            for j in (i*i):i:n # sequence with step i
                isprime[j] = false
            end
        end
    end
    return filter(x -> isprime[x], 1:n) # filter using anonymous function
end
println(es(100)) # print all primes less or equal than 100
@time length(es(10^7)) # check function execution time and memory usage

[2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97]
  1.644036 seconds (10.00 M allocations: 176.738 MB, 2.41% gc time)


664579

## <a name="basicliterals"></a>Basic literals and types

Basic scalar literals (`x::Type` is a literal `x` with type `Type` assertion):

In [20]:
1::Int64 # 64-bit integer, no overflow warnings, fails on 32 bit Julia

1

In [21]:
1.0::Float64 # 64-bit float, defines NaN, -Inf, Inf

1.0

In [22]:
true::Bool # boolean, allows "true" and "false"

true

In [23]:
'c'::Char # character, allows Unicode

'c'

In [24]:
"s"::AbstractString # strings, allows Unicode, see also Strings

"s"

All basic types are immutable. Specifying type assertion is optional (and usually it is not needed, but I give it to show
how you can do it). Type assertions for variables are made in the same way and may improve code performance.

If you do not specify type assertion Julia will choose a default. Note that defaults might be different on 32-bit and
64-bit versions of Julia. A most important difference is for integers which are `Int32` and `Int64` respectively. This means
that `1::Int32` assertion will fail on 64-bit version. Notably `Int` is either `Int64` or `Int32` depending on version (the same
with `UInt`).
There is no automatic type conversion (especially important in function calls). Has to be explicit:

In [25]:
Int64('a') # character to integer

97

In [26]:
Int64(2.0) # float to integer

2

In [27]:
Int64(1.3) # inexact error

LoadError: LoadError: InexactError()
while loading In[27], in expression starting on line 1

In [28]:
Int64("a") # error no conversion possible

LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{Int64}, ::ASCIIString)
This may have arisen from a call to the constructor Int64(...),
since type constructors fall back to convert methods.
Closest candidates are:
  call{T}(::Type{T}, ::Any)
  convert(::Type{Int64}, !Matched::Int8)
  convert(::Type{Int64}, !Matched::UInt8)
  ...
while loading In[28], in expression starting on line 1

In [29]:
Float64(1) # integer to float

1.0

In [30]:
Bool(1) # converts to boolean true

true

In [31]:
Bool(0) # converts to boolean false

false

In [32]:
Bool(2) # conversion error

LoadError: LoadError: InexactError()
while loading In[32], in expression starting on line 1

In [33]:
Char(89) # integer to char

'Y'

In [34]:
string(true) # cast bool to string (works with other types, note small caps)

"true"

In [35]:
string(1,true) # string can take more than one argument and concatenate them

"1true"

In [36]:
zero(10.0) # zero of type of 10.0

0.0

In [37]:
one(Int64) # one of type Int64

1

General conversion can be done using `convert(Type, x)`:

In [ ]:
convert(Int64, 1.0) # convert float to integer

Parsing strings can be done using `parse(Type, str)`:

In [ ]:
parse(Int64, "1") # parse "1" string as Int64

Automatic promotion ofmany arguments to common type (if any) using `promote`:

In [18]:
promote(true, BigInt(1)//3, 1.0) # tuple (see Tuples) of BigFloats, true promoted to 1.0

(1.000000000000000000000000000000000000000000000000000000000000000000000000000000,3.333333333333333333333333333333333333333333333333333333333333333333333333333348e-01,1.000000000000000000000000000000000000000000000000000000000000000000000000000000)

In [19]:
promote("a", 1) # promotion to common type not possible

("a",1)

Many operations (arithmetic, assignment) are defined in a way that performs automatic type promotion.
One can verify type of argument:

In [38]:
typeof("abc") # ASCIIString returned which is a AbstractString subtype

ASCIIString

In [39]:
isa("abc", AbstractString) # true

true

In [40]:
isa(1, Float64) # false, integer is not a float

false

In [41]:
isa(1.0, Float64) # true

true

In [42]:
isa(1.0, Number) # true, Number is abstract type

true

In [43]:
super(Int64) # supertype of Int64

Signed

In [44]:
subtypes(Real) # subtypes of bastract type Real

4-element Array{Any,1}:
 AbstractFloat       
 Integer             
 Irrational{sym}     
 Rational{T<:Integer}

In [48]:
subtypes(Int64)

0-element Array{Any,1}

It is possible to performcalculations using arbitrary precision arithmetic or rational numbers:

In [45]:
BigInt(10)^1000 # big integer

1000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [46]:
BigFloat(10)^1000 # big float, see documentation how to change default precision

9.99999999999999999999999999999999999999999999999999999999999999999999999999968e+999

In [47]:
123//456 # rational numbers are created using // operator

41//152

Type hierarchy of all standard numeric types is given below:

In [57]:
function type_hierarchy(t::DataType, level = 0)
    println(" "^level, t)
    for x in subtypes(t)
        type_hierarchy(x, level+2)
    end
end
type_hierarchy(Number)

Number
  Complex{T<:Real}
  Real
    AbstractFloat
      BigFloat
      Float16
      Float32
      Float64
    Integer
      BigInt
      Bool
      Signed
        Int128
        Int16
        Int32
        Int64
        Int8
      Unsigned
        UInt128
        UInt16
        UInt32
        UInt64
        UInt8
    Irrational{sym}
    Rational{T<:Integer}


## <a name="complexliterals"></a>Complex literals and types
Important types:

In [58]:
Any # all objects are of this type

Any

In [59]:
Union{} # subtype of all types, no object can have this type

Union{}

In [60]:
Void # type indicating nothing, subtype of Any

Void

In [61]:
nothing # only instance of Void

Additionally `#undef` indicates an incompletely initialized instance.

### Tuples
Tuples are immutable sequences indexed from `1`:

In [63]:
() # empty tuple

()

In [64]:
(1,) # one element tuple

(1,)

In [65]:
("a", 1) # two element tuple

("a",1)

In [67]:
('a', false)::Tuple{Char, Bool} # tuple type assertion

('a',false)

In [68]:
x = (1, 2, 3)

(1,2,3)

In [69]:
x[1] # 1 (element)

1

In [70]:
x[1:2] # (1, 2) (tuple)

(1,2)

In [71]:
x[4] # bounds error

LoadError: LoadError: BoundsError: attempt to access (1,2,3)
  at index [4]
while loading In[71], in expression starting on line 1

In [72]:
x[1] = 1 # error - tuple is not mutable

LoadError: LoadError: MethodError: `setindex!` has no method matching setindex!(::Tuple{Int64,Int64,Int64}, ::Int64, ::Int64)
while loading In[72], in expression starting on line 1

In [74]:
a, b = x # tuple unpacking a=1, b=2
println("$a $b")

1 2


### Arrays
Arrays are mutable and passed by reference. Array creation:

In [2]:
Array(Char, 2, 3, 4) # 2x3x4 array of Chars

2x3x4 Array{Char,3}:
[:, :, 1] =
 '\0'  '\0'  '\0'
 '\0'  '\0'  '\0'

[:, :, 2] =
 '\0'  '\0'  '\0'
 '\0'  '\0'  '\0'

[:, :, 3] =
 '\0'  '\0'  '\0'
 '\0'  '\0'  '\0'

[:, :, 4] =
 '\0'  '\0'  '\0'
 '\0'  '\0'  '\0'

In [3]:
Array{Int64}(0, 0) # degenerate 0x0 array of Int64

0x0 Array{Int64,2}

In [4]:
cell(2, 3) # 2x3 array of Any

2x3 Array{Any,2}:
 #undef  #undef  #undef
 #undef  #undef  #undef

In [5]:
zeros(5) # vector of Float64 zeros

5-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0

In [6]:
ones(5) # vector of Float64 ones

5-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0
 1.0

In [7]:
ones(Int64, 2, 1) # 2x1 array of Int64 ones

2x1 Array{Int64,2}:
 1
 1

In [8]:
trues(3), falses(3) # tuple of vector of trues and of falses

(Bool[true,true,true],Bool[false,false,false])

In [9]:
eye(3) # 3x3 Float64 identity matrix

3x3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0

In [10]:
x = linspace(1, 2, 5) # iterator having 5 equally spaced elements

linspace(1.0,2.0,5)

In [11]:
collect(x) # converts iterator to vector

5-element Array{Float64,1}:
 1.0 
 1.25
 1.5 
 1.75
 2.0 

In [12]:
1:10 # iterable from 1 to 10

1:10

In [13]:
1:2:10 # iterable from 1 to 9 with 2 skip

1:2:9

In [14]:
reshape(1:12, 3, 4) # 3x4 array filled with 1:12 values

3x4 Array{Int64,2}:
 1  4  7  10
 2  5  8  11
 3  6  9  12

In [15]:
fill("a", 2, 2) # 2x2 array filled with "a"

2x2 Array{ASCIIString,2}:
 "a"  "a"
 "a"  "a"

In [16]:
repmat(eye(2), 3, 2) # 2x2 identity matrix repeated 3x2 times

6x4 Array{Float64,2}:
 1.0  0.0  1.0  0.0
 0.0  1.0  0.0  1.0
 1.0  0.0  1.0  0.0
 0.0  1.0  0.0  1.0
 1.0  0.0  1.0  0.0
 0.0  1.0  0.0  1.0

In [17]:
x = [1, 2] # two element vector

2-element Array{Int64,1}:
 1
 2

In [18]:
resize!(x, 5) # resize x in place to hold 5 values (filled with garbage)

5-element Array{Int64,1}:
      1
      2
 569348
      2
      2

In [19]:
[1] # vector with one element (not a scalar)

1-element Array{Int64,1}:
 1

In [20]:
[x * y for x in 1:2, y in 1:3] # comprehension generating 2x3 array

2x3 Array{Int64,2}:
 1  2  3
 2  4  6

In [21]:
Float64[x^2 for x in 1:4] # casting comprehension result to Float64

4-element Array{Float64,1}:
  1.0
  4.0
  9.0
 16.0

In [22]:
[1 2] # 1x2 matrix (hcat function)

1x2 Array{Int64,2}:
 1  2

In [23]:
[1 2]' # 2x1 matrix (after transposing)

2x1 Array{Int64,2}:
 1
 2

In [24]:
[1, 2] # vector (vcat function)

2-element Array{Int64,1}:
 1
 2

In [25]:
[1; 2] # vector (hvcat function)

2-element Array{Int64,1}:
 1
 2

In [26]:
[1 2 3; 1 2 3] # 2x3 matrix (hvcat function)

2x3 Array{Int64,2}:
 1  2  3
 1  2  3

In [27]:
[1; 2] == [1 2]' # false, different array dimensions

false

In [28]:
[(1, 2)] # 1-element vector

1-element Array{Tuple{Int64,Int64},1}:
 (1,2)

In [29]:
collect((1, 2)) # 2-element vector by tuple unpacking

2-element Array{Int64,1}:
 1
 2

In [30]:
[[1 2] 3] # append to a row vector (hcat)

1x3 Array{Int64,2}:
 1  2  3

In [31]:
[[1; 2]; 3] # append to a column vector (vcat)

3-element Array{Int64,1}:
 1
 2
 3

Vectors (1D arrays) are treated as column vectors.

Julia offers sparse and distributed matrices (see documentation for details).

Commonly needed array utility functions:

In [32]:
a = [x * y for x in 1:2, y in 1, z in 1:3] # 2x1x3 array of Int64

2x1x3 Array{Int64,3}:
[:, :, 1] =
 1
 2

[:, :, 2] =
 1
 2

[:, :, 3] =
 1
 2

In [33]:
ndims(a) # number of dimensions in a

3

In [34]:
eltype(a) # type of elements in a

Int64

In [35]:
length(a) # number of elements in a

6

In [36]:
size(a) # tuple containing dimension sizes of a

(2,1,3)

vec(a) # cast array to vetor (single dimension)

In [38]:
squeeze(a, 2) # remove 2nd dimension as it has size 1

2x3 Array{Int64,2}:
 1  1  1
 2  2  2

In [39]:
sum(a, 3) # calculate sums for 3rd dimensions, similarly: mean, std, prod, minimum, maximum, any, all

2x1x1 Array{Int64,3}:
[:, :, 1] =
 3
 6

In [40]:
count(x -> x > 0, a) # count number of times a predicate is true, similar: all, any

6

Array access functions:

In [1]:
a = linspace(0, 1) # LinSpace{Float64} of length 50

linspace(0.0,1.0,50)

In [2]:
a[1] # get scalar 0.0

0.0

In [3]:
a[end] # get scalar 1.0 (last position)

1.0

In [4]:
a[1:2:end] # every second element from range, LinSpace{Float64}

linspace(0.0,0.9795918367346939,25)

In [5]:
a[repmat([true, false], 25)] # select every second element, Array{Float64,1}

25-element Array{Float64,1}:
 0.0      
 0.0408163
 0.0816327
 0.122449 
 0.163265 
 0.204082 
 0.244898 
 0.285714 
 0.326531 
 0.367347 
 0.408163 
 0.44898  
 0.489796 
 0.530612 
 0.571429 
 0.612245 
 0.653061 
 0.693878 
 0.734694 
 0.77551  
 0.816327 
 0.857143 
 0.897959 
 0.938776 
 0.979592 

In [6]:
a[[1, 3, 6]] # 1st, 3rd and 6th element of a, Array{Float64,1}

3-element Array{Float64,1}:
 0.0      
 0.0408163
 0.102041 

In [7]:
sub(a, 1:2:50) # view into subsarray of a

25-element SubArray{Float64,1,LinSpace{Float64},Tuple{StepRange{Int64,Int64}},1}:
 0.0      
 0.0408163
 0.0816327
 0.122449 
 0.163265 
 0.204082 
 0.244898 
 0.285714 
 0.326531 
 0.367347 
 0.408163 
 0.44898  
 0.489796 
 0.530612 
 0.571429 
 0.612245 
 0.653061 
 0.693878 
 0.734694 
 0.77551  
 0.816327 
 0.857143 
 0.897959 
 0.938776 
 0.979592 

In [8]:
endof(a) # last index of the collection a

50

Observe the treatment of trailing singleton dimensions:

In [9]:
a = reshape(1:12, 3, 4)

3x4 Array{Int64,2}:
 1  4  7  10
 2  5  8  11
 3  6  9  12

In [10]:
a[:, 1:2] # 3x2 matrix

3x2 Array{Int64,2}:
 1  4
 2  5
 3  6

In [11]:
a[:, 1] # 3-element vector

3-element Array{Int64,1}:
 1
 2
 3

In [12]:
a[1, :] # 1x4 matrix

1x4 Array{Int64,2}:
 1  4  7  10

In [13]:
a[:, :, 1, 1] # works 3x4 matrix

3x4 Array{Int64,2}:
 1  4  7  10
 2  5  8  11
 3  6  9  12

In [14]:
a[:, :, :, [true]] # wroks 3x4x1x1 matrix

3x4x1x1 Array{Int64,4}:
[:, :, 1, 1] =
 1  4  7  10
 2  5  8  11
 3  6  9  12

In [15]:
a[1, 1, [false]] # works 3x4x0 matrix

1x1x0 Array{Int64,3}

Array assignment:

In [17]:
x = reshape(1:8, 2, 4)

2x4 Array{Int64,2}:
 1  3  5  7
 2  4  6  8

In [18]:
x[:, 2:3] = [1 2] # error; size mismatch

LoadError: LoadError: DimensionMismatch("tried to assign 1x2 array to 2x2 destination")
while loading In[18], in expression starting on line 1

In [19]:
x[:, 2:3] = repmat([1 2], 2) # OK

2x2 Array{Int64,2}:
 1  2
 1  2

In [20]:
x[:, 2:3] = 3 # OK

3

Arrays are assigned and passed by reference. Therefore copying is provided:

In [21]:
x = cell(2)
x[1] = ones(2)
x[2] = trues(3)
a = x
b = copy(x) # shallow copy
c = deepcopy(x) # deep copy
x[1] = "Bang"
x[2][1] = false
x

2-element Array{Any,1}:
 "Bang"               
 Bool[false,true,true]

In [22]:
a # identical as x

2-element Array{Any,1}:
 "Bang"               
 Bool[false,true,true]

In [23]:
b # only x[2][1] changed from original x

2-element Array{Any,1}:
 [1.0,1.0]            
 Bool[false,true,true]

In [24]:
c # contents to original x

2-element Array{Any,1}:
 [1.0,1.0]           
 Bool[true,true,true]

Array types syntax examples:

In [25]:
cell(2)::Array{Any, 1} # vector of Any

2-element Array{Any,1}:
 #undef
 #undef

In [26]:
[1 2]::Array{Int64, 2} # 2 dimensional array of Int64

1x2 Array{Int64,2}:
 1  2

In [27]:
[true; false]::Vector{Bool} # vector of Bool

2-element Array{Bool,1}:
  true
 false

In [28]:
[1 2; 3 4]::Matrix{Int64} # matrix of Int64

2x2 Array{Int64,2}:
 1  2
 3  4

### Composite types

Composite types are mutable and passed by reference.
You can define and access composite types:

In [29]:
type Point
    x::Int64
    y::Float64
    meta
end
p = Point(0, 0.0, "Origin")

Point(0,0.0,"Origin")

In [30]:
p.x # access field

0

In [31]:
p.meta = 2 # change field value

2

In [32]:
p.x = 1.5 # error, wrong data type

LoadError: LoadError: InexactError()
while loading In[32], in expression starting on line 1

In [33]:
p.z = 1 # error - no such field

LoadError: LoadError: type Point has no field z
while loading In[33], in expression starting on line 1

In [34]:
fieldnames(p) # get names of instance fields

3-element Array{Symbol,1}:
 :x   
 :y   
 :meta

In [35]:
fieldnames(Point) # get names of type fields

3-element Array{Symbol,1}:
 :x   
 :y   
 :meta

You can define type to be `immutable` by replacing type by immutable. There are also union types (see documentation
for details).

### Dictionaries

Associative collections (key-value dictionaries):

In [36]:
x = Dict{Float64, Int64}() # empty dictionary mapping floats to integers

Dict{Float64,Int64} with 0 entries

In [37]:
y = Dict("a"=>1, "b"=>2) # filled dictionary

Dict{ASCIIString,Int64} with 2 entries:
  "b" => 2
  "a" => 1

In [38]:
y["a"] # element retrieval

1

In [39]:
y["c"] # error

LoadError: LoadError: KeyError: c not found
while loading In[39], in expression starting on line 1

In [40]:
y["c"] = 3 # added element

3

In [41]:
haskey(y, "b") # check if y contains key "b"

true

In [42]:
keys(y), values(y) # tuple of iterators returning keys and values in y

(ASCIIString["c","b","a"],[3,2,1])

In [43]:
delete!(y, "b") # delete key from a collection, see also: pop!

Dict{ASCIIString,Int64} with 2 entries:
  "c" => 3
  "a" => 1

In [44]:
get(y,"c","default") # return y["c"] or "default" if not haskey(y,"c")

3

Julia also supports operations on sets and dequeues, priority queues and heaps (please refer to documentation).

## <a name="strings"></a>Strings

String operations:

In [45]:
"Hi " * "there!" # string concatenation

"Hi there!"

In [46]:
"Ho " ^ 3 # repeat string

"Ho Ho Ho "

In [47]:
string("a= ", 123.3) # create using print function

"a= 123.3"

In [48]:
repr(123.3) # fetch value of show function to a string

"123.3"

In [49]:
contains("ABCD", "CD") # check if first string contains second

true

In [50]:
"\"\n\t\$" # C-like escaping in strings, new \$ escape

"\"\n\t\$"

In [51]:
x = 123
"$x + 3 = $(x+3)" # unescaped $ is used for interpolation

"123 + 3 = 126"

In [52]:
"\$199" # to get a $ symbol you must escape it

"\$199"

PCRE regular expressions handling:

In [53]:
r = r"A|B" # create new regexp

r"A|B"

In [54]:
ismatch(r, "CD") # false, no match found

false

In [55]:
m = match(r, "ACBD") # find first regexp match, see documentation for details

RegexMatch("A")

There is a vast number of string functions—please refer to documentation.

## <a name="programming"></a>Programming constructs

The simplest way to create new variable is by assignment:

In [56]:
x = 1.0 # x is Float64

1.0

In [57]:
x = 1 # now x is Int32 on 32 bit machine and Int64 on 64 bit machine

1

In [58]:
y::Float64 = 1.0 # y must be Float64; in global scope performs assertion on y type when it exists

LoadError: LoadError: TypeError: typeassert: expected Float64, got Dict{ASCIIString,Int64}
while loading In[58], in expression starting on line 1

In [60]:
y # value of y from eariler computations

Dict{ASCIIString,Int64} with 2 entries:
  "c" => 3
  "a" => 1

Expressions can be compound using ; or begin end block:

In [61]:
x = (a = 1; 2 * a) # after: x = 2; a = 1
(x, a)

(2,1)

In [62]:
y = begin
    b = 3
    3 * b
end # after: y = 9; b = 3
(y, b)

(9,3)

There are standard programming constructs:

In [63]:
if false # if clause requires Bool test
    z = 1
elseif 1==2
    z = 2
else
    a = 3
end # after this a = 3 and z is undefined
(a, isdefined(:z))

(3,false)

In [64]:
1==2 ? "A" : "B" # standard ternary operator

"B"

In [65]:
i = 1
while true
    i += 1
    if i > 10
        break
    end
end
i

11

In [66]:
for x in 1:10 # x in collection, can also use = here instead of in
    if 3 < x < 6
        continue # skip one iteration
    end
    println(x)
end # x is introduced in loop outer scope
x

1
2
3
6
7
8
9
10


10

You can define your own functions:

In [67]:
f(x, y = 10) = x + y # new function f with y defaulting to 10; last result returned
f(3, 2) # simple call, 5 returned

5

In [68]:
f(3) # 13 returned

13

In [69]:
function g(x::Int, y::Int) # type restriction
return y, x # explicit return of a tuple
end
g(x::Int, y::Bool) = x * y # add multiple dispatch
g(2, true) # second definition is invoked

2

In [70]:
methods(g) # list all methods defined for g

# 2 methods for generic function "g":
g(x::Int64, y::Int64) at In[69]:2
g(x::Int64, y::Bool) at In[69]:4

In [71]:
(x -> x^2)(3) # anonymous function with a call

9

In [72]:
() -> 0 # anonymous function with no arguments

(anonymous function)

In [73]:
h(x...) = sum(x)/length(x) - mean(x) # vararg function; x is a tuple
h(1, 2, 3) # result is 0

0.0

In [74]:
x = (2, 3) # tuple
f(x) # error

LoadError: LoadError: MethodError: `+` has no method matching +(::Tuple{Int64,Int64}, ::Int64)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...)
  +(!Matched::Int64, ::Int64)
  +(!Matched::Complex{Bool}, ::Real)
  ...
while loading In[74], in expression starting on line 2

In [75]:
f(x...) # OK - tuple unpacking

5

In [76]:
s(x; a = 1, b = 1) = x * a / b # function with keyword arguments a and b
s(3, b = 2) # call with keyword argument

1.5

In [77]:
t(; x::Int64 = 2) = x # single keyword argument
t() # 2 returned

2

In [78]:
t(; x::Bool = true) = x # no multiple dispatch for keyword arguments; function overwritten
t() # true; old function was overwritten

true

In [79]:
q(f::Function, x) = 2 * f(x) # simple function wrapper
q(x -> 2x, 10) # 40 returned, no need to use * in 2x (means 2*x)

40

In [80]:
q(10) do x # creation of anonymous function by do construct, useful eg. in IO
    2 * x
end

40

In [81]:
m = reshape(1:12, 3, 4)
map(x -> x ^ 2, m) # 3x4 array returned with transformed data

3x4 Array{Int64,2}:
 1  16  49  100
 4  25  64  121
 9  36  81  144

In [83]:
filter(x -> bits(x)[end] == '0', 1:12) # a fancy way to choose even integers from the range

6-element Array{Int64,1}:
  2
  4
  6
  8
 10
 12

As a convention functions with name ending with ! change their arguments in-place. See for example `resize!` in this
document.
    
Default function argument beasts:

In [84]:
y = 10
f1(x=y) = x; f1() # 10

10

In [85]:
f2(x=y,y=1) = x; f2() # 10

10

In [86]:
f3(y=1,x=y) = x; f3() # 1

1

In [87]:
f4(;x=y) = x; f4() # 10

10

In [88]:
f5(;x=y,y=1) = x; f5() # error - y not defined yet :(

LoadError: LoadError: UndefVarError: y not defined
while loading In[88], in expression starting on line 1

In [89]:
f6(;y=1,x=y) = x; f6() # 1

1

## <a name="scoping"></a>Variable scoping

The following constructs introduce new variable scope: `function`, `while`, `for`, `try`/`catch`, `let`, `type`.

You can define variables as:

* `global`: use variable from global scope;
* `local`: define new variable in current scope;
* `const`: ensure variable type is constant (global only).

Special cases:

In [91]:
w # error, variable does not exist

LoadError: LoadError: UndefVarError: w not defined
while loading In[91], in expression starting on line 1

In [92]:
f() = global w = 1
f() # after the call w is defined globally
w

1

In [93]:
function f1(n)
    x = 0
    for i = 1:n
        x = i
    end
    x
end
f1(10) # 10; inside loop we use outer local variable

10

In [94]:
function f2(n)
    x = 0
    for i = 1:n
        local x
        x = i
    end
    x
end
f2(10) # 0; inside loop we use new local variable

0

In [95]:
function f3(n)
    for i = 1:n
        local x # this local can be omitted; for introduces new scope
        x = i
    end
    x
end
f3(10) # x fetched from global scope as it wase already defined

(2,3)

In [103]:
const n = 2
n = 3 # warning, value changed

3

In [104]:
n = 3.0 # error, wrong type

LoadError: LoadError: invalid redefinition of constant n
while loading In[104], in expression starting on line 1

In [105]:
function fun() # no warning
    const x = 2
    x = true
end
fun() # true, no warning

true

Global constants speed up execution.

The `let` rebinds the variable:

In [106]:
Fs = cell(2)
i = 1
while i <= 2
    j = i
    Fs[i] = () -> j
    i += 1
end
Fs[1](), Fs[2]() # (2, 2); the same binding for j

(2,2)

In [108]:
Fs = cell(2)
i = 1
while i <= 2
    let j = i
        Fs[i] = () -> j
    end
    i += 1
end
Fs[1](), Fs[2]() # (1, 2); new binding for j

(1,2)

In [109]:
Fs = cell(2)
i = 1
for i in 1:2
    j = i
    Fs[i] = () -> j
end
Fs[1](), Fs[2]() # (1, 2); for loops and comprehensions rebind variables

(1,2)

## <a name="modules"></a>Modules

Modules encapsulate code. Can be reloaded, which is useful to redefine functions and types, as top level functions
and types are defined as constants.

In [112]:
module M # module name; can be replaced in one session
export xx # what module exposes for the world
xx = 1
y = 2 # hidden variable
end

whos(M) # list exported variables

                             M    871 bytes  Module
                            xx      8 bytes  Int64


In [113]:
xx # not found in global scope

LoadError: LoadError: UndefVarError: xx not defined
while loading In[113], in expression starting on line 1

In [114]:
M.y # direct variable access possible

2

In [115]:
# import all exported variables
# load standard packages this way
using M

#import variable y to global scope (even if not exported)
import M.y